<a href="https://colab.research.google.com/github/danielhalak/dl_project_spices_richness_mamba/blob/main/updated_dl_project_mamba_daniel_last_one_8_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm

ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.3-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2

In [ ]:
import os
import sys
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/dl_project_data/sscape-avian-div-generalisability
sys.path.append('/content/drive/MyDrive/dl_project_data/sscape-avian-div-generalisability')


ValueError: mount failed

In [ ]:
import torch
from torch.utils.data import Dataset
import glob
def encode_birds(entry, bird_to_index, num_birds ):
    one_hot_vector = np.zeros(num_birds, dtype=np.float32)
    for bird in entry:
        if bird in bird_to_index:  # Check to avoid errors with unexpected bird names
            one_hot_vector[bird_to_index[bird]] = 1.0
        else:
          print(f"dont have {bird}")
    if(max(one_hot_vector)==0):
      print(f"bad entry {entry}")
    else:
      return one_hot_vector

parsed_data_dir = '../npy/'
npy_files_safe = [os.path.splitext(os.path.basename(f))[0] for f in glob.glob(f"{parsed_data_dir}/*safe*.npy")]
npy_files = ['india_vggish', 'us_vggish', 'taiwan_vggish']  # Add all region files
npy_files.extend(npy_files_safe)
for dataset in npy_files:
    print(dataset)
avi = []
for dataset in npy_files:
    parsed_vggish_path = os.path.join(parsed_data_dir, '{}.npy'.format(dataset))
    all_vggish_pcs = np.load(parsed_vggish_path, allow_pickle=True)
    all_vggish_pcs = np.asarray(all_vggish_pcs)

    all_vggish_pc_ids = np.asarray([pc.id for pc in all_vggish_pcs])
    for pc in all_vggish_pcs:
        avi.extend(pc.avi_spec_comm)
avi_set = set(avi)
print(avi_set)
bird_to_index = {bird: idx for idx, bird in enumerate(sorted(avi_set))}

def decode_birds(one_hot_vector, bird_to_index):
    # Inverse the bird_to_index mapping to get the bird name from index
    index_to_bird = {idx: bird for bird, idx in bird_to_index.items()}

    # Get the indices of the birds that are present in the one-hot vector
    bird_indices = np.where(one_hot_vector == 1)[0]

    # Get the bird names from the indices
    # Check if predicted index exists before accessing
    birds = [index_to_bird[idx] for idx in bird_indices if idx in index_to_bird]

    return birds

Generating a encoder dicoder for number of birds species to decrease the size of the output vector:
Need to be trained only once, then the weights will be loaded each time

In [ ]:
"""

import torch
import torch.nn as nn
import torch.optim as optim

class LabelAutoencoder(nn.Module):
    def __init__(self, input_dim=200, latent_dim=32):
        super(LabelAutoencoder, self).__init__()

        # Encoder: Compress input (200D → 64D → 16D)
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),  # Reduce from 200D → 64D
            nn.ReLU(),
            nn.Linear(64, latent_dim)  # Further reduce to latent space (32D)
        )

        # Decoder: Expand latent space back to original size (32D → 64D → 200D)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        latent = self.encoder(x)  # Encode input to latent space
        reconstructed = self.decoder(latent)  # Decode back to original size
        return latent, reconstructed

    def decode(self, x):
      return self.decoder(x)
"""

In [ ]:
"""
def train_decoder(train_dataset, input_dim = 200, latent_dim = 32, batch_size = 256, num_epochs = 1):
  # Set dimensions
    # number of birds species
   # Compressed size

  # Initialize model
  autoencoder = LabelAutoencoder(input_dim, latent_dim)

  # Define loss function (Mean Squared Error)
  criterion = nn.BCEWithLogitsLoss()

  # Define optimizer
  optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)  # Learning rate of 0.001

  # Number of training iterations

  for epoch in range(num_epochs):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for target in dataloader:
        # Reset gradients
        optimizer.zero_grad()

        # Forward pass: Encode → Decode
        latent, reconstructed = autoencoder(target[0])

        # Compute reconstruction loss
        loss = criterion(reconstructed, target[0])

        # Backpropagation: Update weights
        loss.backward()
        optimizer.step()

        # Print loss every 10 epochs
    if epoch % 2 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Save the trained autoencoder
  torch.save(autoencoder.state_dict(), "label_autoencoder.pth")
  """

In [ ]:
"""
min_ones, max_ones = 1, 40
num_samples = 100000
input_dim = len(avi_set)

# Create dataset
dataset = torch.zeros(num_samples, input_dim)

for i in range(num_samples):
    num_ones = torch.randint(min_ones, max_ones + 1, (1,)).item()  # Random number of ones
    indices = torch.randperm(input_dim)[:num_ones]  # Random positions
    dataset[i, indices] = 1  # Set selected positions to 1


from torch.utils.data import DataLoader, TensorDataset


dataset = TensorDataset(dataset,)
train_decoder(dataset, input_dim = len(avi_set), num_epochs=50)
"""

In [ ]:

class VGGishRegionDataset(Dataset):
    def __init__(self, all_datasets, decoder = ""):
        """
        Args:
            npy_files (list): List of paths to `.npy` files for different regions.
        """
        self.data = []  # Store all (pc_id, audio_feats, avi_spec_comm)


        # Load and process each region
        for dataset in all_datasets:
            parsed_vggish_path = os.path.join(parsed_data_dir, '{}.npy'.format(dataset))
            all_vggish_pcs = np.load(parsed_vggish_path, allow_pickle=True)
            all_vggish_pcs = np.asarray(all_vggish_pcs)
            if(decoder==""):
              all_vggish_pc_ids = np.asarray([pc.id for pc in all_vggish_pcs])
              for pc in all_vggish_pcs:
                  if(len(pc.avi_spec_comm)>0):
                    self.data.append((pc.id, pc.audio_feats, encode_birds(pc.avi_spec_comm, bird_to_index, len(avi_set))))
                    #revesed_data = pc.audio_feats[::-1].copy()
                    #self.data.append((pc.id + "_reversed", revesed_data, encode_birds(pc.avi_spec_comm, bird_to_index, len(avi_set))))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pc_id, audio_feats, avi_spec_comm = self.data[idx]
        audio_feats = torch.tensor(audio_feats, dtype=torch.float32)

        if avi_spec_comm is not None:
            avi_spec_comm = torch.tensor(avi_spec_comm, dtype=torch.float32)
            return audio_feats, avi_spec_comm, pc_id
        else:
            return audio_feats, pc_id


In [ ]:
from torch.nn.utils.rnn import pad_sequence
#At the Beginning
def pad_at_beginning(sequences, padding_value=0.0):
    max_len = max(seq.shape[0] for seq in sequences)  #changed because need standartization for pos_embeding
    #max_len=1700
    padded_seqs = []
    for seq in sequences:
        pad_size = max_len - seq.shape[0]
        if(pad_size<0):
          print("error")
        padding = torch.full((pad_size,) + seq.shape[1:], padding_value, dtype=seq.dtype)  # Handles multi-dim tensors
        padded_seqs.append(torch.cat([padding, seq], dim=0))  # Concatenating padding at the beginning

    return torch.stack(padded_seqs)

def collate_fn(batch):
    """
    Custom collate function to pad variable-length sequences.
    """
    if len(batch[0]) == 3:  # If `avi_spec_comm` is included
        sequences, avi_spec_comm, pc_ids = zip(*batch)
        avi_spec_comm = torch.stack(avi_spec_comm)
    else:
        sequences, pc_ids = zip(*batch)
        avi_spec_comm = None

    # Pad sequences dynamically
    padded_sequences = pad_at_beginning(sequences, padding_value=0.0)

    return padded_sequences, avi_spec_comm, pc_ids


Regular data generation:


Data generator thats generates the same data every time:

In [ ]:
import os
import glob
import torch
import numpy as np
from torch.utils.data import DataLoader, random_split

# Fix randomness for reproducibility
torch.manual_seed(42)  # 🔹 Fix PyTorch randomness
np.random.seed(42)      # 🔹 Fix NumPy randomness

# If using CUDA, ensure deterministic behavior (optional)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# List of region `.npy` files
parsed_data_dir = '../npy/'
npy_files_safe = [os.path.splitext(os.path.basename(f))[0] for f in glob.glob(f"{parsed_data_dir}/*safe*.npy")]
npy_files = ['india_vggish', 'us_vggish', 'taiwan_vggish']  # Add all region files
npy_files.extend(npy_files_safe)

# Initialize dataset
dataset = VGGishRegionDataset(npy_files)

# Split sizes
train_size = int(0.8 * len(dataset))
valid_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - valid_size  # Ensure all data is used

# 🔹 Ensure deterministic train/valid/test split
split_generator = torch.Generator().manual_seed(42)  # Same generator for consistent split
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size], generator=split_generator)

print(f"Train size: {len(train_dataset)}, Valid size: {len(valid_dataset)}, Test size: {len(test_dataset)}")

# 🔹 Create a generator for DataLoaders to ensure deterministic shuffling
dataloader_generator = torch.Generator().manual_seed(42)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, generator=dataloader_generator)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, generator=dataloader_generator)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn, generator=dataloader_generator)

print(len(train_loader))
print(len(valid_loader))

# Test a batch from the train loader
for batch in train_loader:
    inputs, _, ids = batch
    print("Train Batch shape:", inputs.shape)  # (batch_size, max_seq_len_in_batch, feature_dim)
    print("PC IDs:", ids)
    break


In [ ]:
for t, i , l in train_loader:
  print(t.shape)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from mamba_ssm import Mamba
import torch.nn.init as init


class MambaBirdClassifier(nn.Module):
    def __init__(
        self,
        device,
        input_dim=128,  # AVGGish embedding dimension
        hidden_dim=256,
        n_mamba_layers=3,
        n_classes=200,  # Number of bird species
        dropout=0.3,
        seq_len=1000,
        num_skip_connections=4
    ):
        super().__init__()
        self.device = device

        # increase the dimensionality so the network has more parameters to learn
        self.input_projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU()
        )

        # Positional embedding (ensuring it's on the correct device)
        #self.pos_embedding = nn.Parameter(torch.randn(1, seq_len, hidden_dim, device=self.device) * 0.02)

        # Mamba layers
        self.mamba_layers = nn.ModuleList([nn.Sequential(
            Mamba(
                d_model=hidden_dim,       # Model dimension
                d_state=256,               # SSM state expansion factor, responsible for long-term memory (matrix B)
                d_conv=4,                 # Local convolution width, short-term memory (how much states can we see backward)
                expand=2,                 # Expansion factor for feed-forward block
                dt_min=0.001,             # Min step size
                dt_max=0.1,               # Max step size
                dt_init="random",         # Initialization of step sizes
                dt_scale=1.0,             # Scaling factor for step sizes
                dt_init_floor=1e-4,       # Minimum value for dt initialization
                bias=False                # Whether to learn bias
            ),
            nn.Dropout(dropout),
            nn.LayerNorm(hidden_dim)) for _ in range(n_mamba_layers)])


        # Skip connection logic
        step = seq_len // num_skip_connections
        #self.skip_seq = torch.tensor(np.arange(start=step, stop=seq_len, step=step), dtype=torch.long, device=self.device)

        # Skip connection weights and linear transformations
        #self.skip_weights = nn.ParameterList([nn.Parameter(torch.ones(1, device=self.device)) for _ in range(len(self.skip_seq))])
        #self.linear_trans = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(len(self.skip_seq))])

        # Layer norm after Mamba layers
        self.ln_f = nn.LayerNorm(hidden_dim)

        # Global pooling and classification head
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]
        x = x.to(self.device)

        # Project input to higher dimension, so the network can learn better how much lo weight in every kind of input
        x = self.input_projection(x)
        x= self.dropout(x)

        # Add positional embeddings
        #x = x + self.pos_embedding
        #print("before mamba", x.shape)

        # Apply Mamba layers
        for mamba_layer in self.mamba_layers:
            x = mamba_layer(x)

        # Apply skip connections
        """
        for i, s in enumerate(self.skip_seq):
            x_new = x.clone()
            x_new[:, -1, :] = (self.skip_weights[i] * x[:, s, :] + x[:, -1, :]).clone()
            x = x_new
            """
        #print("after skip connection", x.shape)
        output =  x[:, -1, :]
        # Final normalization #daniel: i dont know why we put the normalization layer here and not somewhere else
        #output = self.ln_f(output)


        # Global pooling (convert to [batch_size, hidden_dim, seq_len] for pooling)
        #output = output.transpose(1, 2)
        #output = self.pool(output).squeeze(-1)
        #print("before classification", output.shape)

        # Classification
        output = self.dropout(output)
        logits = self.classifier(output)
        #print("after classification", logits.shape)


        return logits

def init_weights(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)  # Xavier initialization
        if m.bias is not None:
            init.zeros_(m.bias)  # Set bias to zero
    elif isinstance(m, nn.LayerNorm):
        init.ones_(m.weight)  # Set LayerNorm scale to 1
        init.zeros_(m.bias)   # Set bias to zero
    elif isinstance(m, Mamba):
        pass
# Instantiate model and move to the correct device
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
print(model)



In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Assuming `avi_set` is your list of all possible birds (labels)

# To get more from our dataset - we gave each class weights for each label in avi_set - so labels with less representations will have more weights in the model
class_weights = compute_class_weight('balanced', classes=np.unique(avi), y=avi)
class_weight_dict = dict(zip(np.unique(avi), class_weights))
print(class_weight_dict)




In [ ]:
def create_weights_from_train_loader(train_loader, num_birds):
    """
    Creates weights based on bird frequencies in a train loader
    where labels are already one-hot encoded. Computes both:
    - Classification weights (per class, for BCE loss)
    - Count weights (per total count, for L1 loss)

    Args:
        train_loader: The DataLoader containing training data with one-hot encoded labels
        num_birds: Total number of bird classes

    Returns:
        class_weights: Tensor of weights inversely proportional to class frequency
        count_weights: Tensor of weights inversely proportional to count frequency
    """
    # Initialize count for each bird class
    bird_counts = torch.zeros(num_birds)
    count_distribution = {}  # To store count frequencies
    total_samples = 0

    print("Counting bird occurrences in train loader...")
    for batch in train_loader:
        labels = batch[1]  # Assuming labels are one-hot encoded
        batch_size = labels.size(0)
        total_samples += batch_size

        # Sum occurrences of each bird class
        bird_counts += torch.sum(labels, dim=0)

        # Compute total bird counts per sample
        sample_counts = torch.sum(labels, dim=1).cpu().numpy()  # Total birds per sample

        # Track the frequency of each unique count
        for count in sample_counts:
            count_distribution[count] = count_distribution.get(count, 0) + 1

    # Compute classification weights (inverse frequency)
    epsilon = 1e-5  # Small value to avoid division by zero
    class_weights = 1.0 / (bird_counts + epsilon)
    class_weights = class_weights / torch.sum(class_weights) * num_birds
    class_weights = torch.clamp(class_weights, max=5.0)  # Cap extreme values

    # Compute L1 count weights (inverse frequency)
    max_count = max(count_distribution.keys()) if count_distribution else 1
    max_count = int(max_count)
    count_weights = torch.zeros((max_count + 1))  # Ensure space for all counts

    for count, freq in count_distribution.items():
        print(f"count {count} freq {freq}")
        count_weights[int(count)] = 1.0 / (freq + epsilon)  # Inverse frequency

    # Normalize count weights
    count_weights = count_weights / torch.sum(count_weights) * len(count_distribution)
    count_weights = torch.clamp(count_weights, max=5.0)  # Cap extreme values

    print(f"Generated class weights and count weights.")

    return class_weights, count_weights


# Usage example
num_birds = len(bird_to_index)
class_weights, count_weights = create_weights_from_train_loader(train_loader, len(avi_set))


In [ ]:
def weighted_focal_loss(probs, targets, class_weights, gamma=2.0, fn_penalty=2.0):
    """
    Focal loss to address class imbalance, with weighting and false negative penalty.

    Args:
        probs: Sigmoid probabilities [batch_size, n_classes]
        targets: Binary targets [batch_size, n_classes]
        class_weights: Per-class weights [n_classes]
        gamma: Focusing parameter (higher means more focus on hard examples)
        fn_penalty: Multiplier for false negative errors (higher means more penalty)
    """
    # Convert to device of other tensors
    class_weights = class_weights.to(probs.device)

    # Calculate focal loss components
    p_t = probs * targets + (1 - probs) * (1 - targets)
    ce_loss = -torch.log(p_t + 1e-8)  # Add small epsilon to prevent log(0)

    # Calculate the base focal loss
    focal = (1 - p_t) ** gamma * ce_loss

    # Create a mask for false negatives (where target=1 but prediction is low)
    false_neg_mask = targets * (1 - probs)

    # Apply the false negative penalty
    # This increases the loss for false negatives by the fn_penalty factor
    penalty_multiplier = 1.0 + (fn_penalty - 1.0) * false_neg_mask

    # Apply both class weights and false negative penalty
    weighted_focal = focal * class_weights.unsqueeze(0) * penalty_multiplier

    return weighted_focal.mean()

In [ ]:
def false_positive_negative(target, preds):
  hit = 0
  fp = 0
  fn = 0
  for i,t in enumerate(target):
    for j, r in enumerate(t):
      if(preds[i][j]==target[i][j]):
        hit+=1
      if(preds[i][j]==1 and target[i][j]==0):
        fn+=1
      if(preds[i][j]==0 and target[i][j]==1):
        fp+=1
  return hit, fp, fn


In [ ]:
def binary_output(x):
    return torch.sigmoid(x)   # maybe add (x > 0.5).float()
def calculate_y_mean(loader):
    sum = 0
    count = 0
    for _, l, _ in loader:
      s =  torch.sum(l, dim=1)
      sum +=torch.sum(s)
      count += len(s)
    return sum/count

def calculate_r2(y_pred, y_true, y_mean):
  sum_distances_mean = torch.sum((y_mean-y_true)**2)
  sum_distances_pred = torch.sum((y_pred-y_true)**2)
  return sum_distances_mean, sum_distances_pred


def train_mamba(model, train_loader, batch_size, vector_dim, device, optimizer, regular_bce, scheduler, train_mean =0, loss_type_weight = 0.6):
    """
    Train the model on a given dataset.

    Args:
        model: The trained model to evaluate.
        train_loader: DataLoader for the training dataset.
        batch_size (int): Batch size for evaluation.
        vector_dim (int): Dimensionality of the output vectors.
        device: The device (CPU/GPU) to run the evaluation on.

    Returns:
        float: The average loss over the training epoch dataset.

    """
    #init
    batch, dim = batch_size, vector_dim
    #Start model training
    model.train()
    loss_epoch = 0
    # Modify the loss function to include class weights
    class_weights_tensor = torch.tensor(data=class_weights, dtype=torch.float32).to(device)
    if(regular_bce):
      bce_loss_birds = nn.BCEWithLogitsLoss(class_weights_tensor)
    mse_loss_count = nn.MSELoss()
    correct_predictions = 0
    correct_count_predictions=0
    total_samples = 0
    i = 0
    FP = 0
    FN = 0
    hit = 0
    madad = 0
    pred_cnt = 0
    total_gueses=0
    r2_pred =0
    r2_mean=0
    for (feats,target, ids) in train_loader:
      target = target.to(device)
      targets_count = torch.sum(target, dim=1)
      logits = model(feats.to(device))
      logits_count = torch.sum(binary_output(logits), dim=1)
      # Convert class weights into a tensor
      if(regular_bce==False):
        probs = torch.sigmoid(logits)
        classification_loss = weighted_focal_loss(
            probs=probs,
            targets=target,
            class_weights=class_weights,
            gamma=2,
            fn_penalty=2
        )
      else:
        classification_loss=  bce_loss_birds(logits, target)
      loss = loss_type_weight*classification_loss + (1-loss_type_weight)*mse_loss_count(logits_count, targets_count)
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      # Compute the loss
      loss_epoch += loss.item()
      # Compute accuracy (assumes binary classification for BCE loss)
      sum_distances_mean, sum_distances_pred = calculate_r2(logits_count, targets_count, train_mean)
      r2_pred+=sum_distances_pred
      r2_mean+=sum_distances_mean
      preds = (binary_output(logits) > 0.5).float()  # Convert logits to binary predictions
      correct_predictions += (preds == target).sum().item()
      logits_count_for_acc = torch.sum(preds, dim=1)
      correct_count_predictions +=(logits_count_for_acc==targets_count).sum().item()
      total_samples += target.shape[0]
      madad += torch.sum(torch.abs(targets_count-logits_count)/targets_count) #NEED TO ADD - COUNT ERROR PERCENT - HOW FAR THE AMOUNT OF BIRDS WE COUND FROM THE AMOUNT OF ACTUAL COUNT TARGET
      preds_np = preds.cpu().numpy()
      target_np = target.cpu().numpy()
      count_positives =false_positive_negative(target, preds)
      hit+=count_positives[0]
      FP+=count_positives[1]
      FN+=count_positives[2]
      # Print example predictions
      for i_pred in range(len(preds_np)):
          decoded_pred = decode_birds(preds_np[i_pred], bird_to_index)  # Decoded prediction
          decoded_target = decode_birds(target_np[i_pred], bird_to_index)  # Decoded target
          if(i==103) and (i_pred==0):
              print(f"Item {i_pred}:")
              print(f"Predicted Birds: {decoded_pred}")
              print(f"Target Birds: {decoded_target}")


      #count how many of the target birds were identified
          for j in decoded_target:
            if j in decoded_pred:
              pred_cnt = pred_cnt + 1
      #print the identified and target bird names
      if (i == 103):
        #print("target count: ", targets_count)
        #print("logits count: ", logits_count_for_acc)
        print("number of target birds that where identified: ", pred_cnt, "out of ", torch.sum(logits_count_for_acc), "number of targert birds")
      total_gueses +=torch.sum(logits_count_for_acc)
      i = i+1
    avg_loss = target.shape[0] * loss_epoch / len(train_loader)
    correct = {"avg_loss": avg_loss,
                "bird classifier":correct_predictions/(total_samples*target.shape[1]),
               "count":correct_count_predictions/total_samples,
               "count_error": madad/total_samples,
               "idf_perc":pred_cnt /(total_gueses + 1e-8),
               "r2":  1-(r2_pred/ (r2_mean + 1e-8)),
               "fp": FP/total_samples,
               "fn": FN/total_samples}
    return model, avg_loss, correct


In [ ]:
def eval_mamba(model, eval_loader, batch_size, vector_dim, device,  val_mean =0, loss_type_weight = 0.6):
    """
    Evaluate the model on a given dataset.

    Args:
        model: The trained model to evaluate.
        eval_loader: DataLoader for the evaluation dataset.
        batch_size (int): Batch size for evaluation.
        vector_dim (int): Dimensionality of the output vectors.
        device: The device (CPU/GPU) to run the evaluation on.

    Returns:
        float: The average loss over the evaluation dataset.
        float: The accuracy of the model (if applicable).
    """
    # Initialize evaluation mode
    model.eval()

    # Convert class weights into a tensor
    #class_weights_tensor = torch.tensor([class_weight_dict.get(bird, 1.0) for bird in avi_set], dtype=torch.float32).to(device)

    # Modify the loss function to include class weights
    class_weights_tensor = torch.tensor(data=class_weights, dtype=torch.float32).to(device)
    bce_loss_birds = nn.BCEWithLogitsLoss(class_weights_tensor)
    mse_loss_count = nn.MSELoss()
    correct_predictions = 0
    correct_count_predictions=0
    total_loss = 0
    total_samples = 0
    i = 0
    madad = 0
    hit=0
    FP=0
    FN=0
    pred_cnt = 0
    total_gueses=0
    r2_pred =0
    r2_mean=0
    # Disable gradient computation for evaluation
    with torch.no_grad():
        for (feats,target, ids) in eval_loader:
            total_count =0
            total_predicted_count =0
            target = target.to(device)
            targets_count = torch.sum(target, dim=1)
            logits = model(feats.to(device))
            logits_count = torch.sum(binary_output(logits), dim=1)
            # Convert class weights into a tensor
            loss_birds = bce_loss_birds(logits, target)
            loss_count = mse_loss_count(logits_count, targets_count)
            loss = loss_type_weight*loss_birds + (1-loss_type_weight)*loss_count
            # Compute the loss
            total_loss += loss.item()
            sum_distances_mean, sum_distances_pred = calculate_r2(logits_count, targets_count, val_mean)
            r2_pred+=sum_distances_pred
            r2_mean+=sum_distances_mean
            preds = (binary_output(logits) > 0.5).float()  # Convert logits to binary predictions
            count_positives =false_positive_negative(target, preds)
            hit+=count_positives[0]
            FP+=count_positives[1]
            FN+=count_positives[2]
            correct_predictions += (preds == target).sum().item()
            logits_count_for_acc = torch.sum(preds, dim=1)
            correct_count_predictions +=(logits_count_for_acc==targets_count).sum().item()
            total_samples += target.shape[0]
            madad += torch.sum(torch.abs(targets_count-logits_count)/targets_count) #NEED TO ADD - COUNT ERROR PERCENT - HOW FAR THE AMOUNT OF BIRDS WE COUND FROM THE AMOUNT OF ACTUAL COUNT TARGET
            preds_np = preds.cpu().numpy()
            target_np = target.cpu().numpy()
            # Print example predictions
            for i_pred in range(len(preds_np)):
                decoded_pred = decode_birds(preds_np[i_pred], bird_to_index)  # Decoded prediction
                decoded_target = decode_birds(target_np[i_pred], bird_to_index)  # Decoded target
                if(i==5) and (i_pred==0):
                    print(f"Item {i_pred}:")
                    print(f"Predicted Birds: {decoded_pred}")
                    print(f"Target Birds: {decoded_target}")
            #count how many of the target birds were identified
                for j in decoded_target:
                  if j in decoded_pred:
                    pred_cnt = pred_cnt + 1
            #print the identified and target bird names
            if (i == 10):
              print("target count: ", targets_count)
              print("logits count: ", logits_count_for_acc)
              print("number of target birds that where identified: ", pred_cnt, "out of ", torch.sum(logits_count_for_acc), "number of targert birds")
            total_gueses +=torch.sum(logits_count_for_acc)
            i = i+1
        avg_loss = target.shape[0] * total_loss / len(eval_loader)
        correct = {"avg_loss": avg_loss,
                "bird classifier":correct_predictions/(total_samples*target.shape[1]),
               "count":correct_count_predictions/total_samples,
               "count_error": madad/total_samples,
               "idf_perc":pred_cnt /(total_gueses + 1e-8),
               "r2":  1-(r2_pred/ (r2_mean + 1e-8)),
               "fp": FP/(total_samples*target.shape[1]),
               "fn": FN/(total_samples*target.shape[1])}
        return model, avg_loss, correct

Function for printing the graphs:

In [ ]:
import matplotlib.pyplot as plt

def plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid=None, title=""):
    """
    Plots idf_perc and count_error over epochs, indicating changes in learning rate and alpha.
    If correct_valid is provided, it also plots the validation metrics.

    Parameters:
        correct_train (list of dict): List where each dictionary contains
        'learning_rate', 'alpha', 'idf_perc', and 'count_error'.
        correct_valid (list of dict, optional): Same format as correct_train, but for validation.
    """
    epochs = list(range(1, len(correct_train) + 1))
    idf_perc_train = [entry['idf_perc'].detach().cpu().numpy() if hasattr(entry['idf_perc'], 'cpu') else entry['idf_perc'] for entry in correct_train]
    count_error_train = [entry['count_error'].detach().cpu().numpy() if hasattr(entry['count_error'], 'cpu') else entry['count_error'] for entry in correct_train]
    r2 = [entry['r2'].detach().cpu().numpy() if hasattr(entry['count_error'], 'cpu') else entry['count_error'] for entry in correct_train]
    learning_rates = [entry['learning_rate'] for entry in correct_train]
    alphas = [entry['alpha'] for entry in correct_train]

    if correct_valid:
        idf_perc_valid = [entry['idf_perc'].detach().cpu().numpy() if hasattr(entry['idf_perc'], 'cpu') else entry['idf_perc'] for entry in correct_valid]
        count_error_valid = [entry['count_error'].detach().cpu().numpy() if hasattr(entry['count_error'], 'cpu') else entry['count_error'] for entry in correct_valid]
        r2_valid = [entry['r2'].detach().cpu().numpy() if hasattr(entry['count_error'], 'cpu') else entry['count_error'] for entry in correct_valid]

    fig, axes = plt.subplots(3, 1, figsize=(10, 8))

    # Function to annotate parameter changes
    def annotate_changes(values, axis, label):
        color = "gray" if label == "Alpha" else "black"
        prev_value = values[0]
        for i, value in enumerate(values):
            if value != prev_value:
                axis.axvline(i + 1, color=color, linestyle='--', alpha=0.6)
                axis.text(i + 1, max(axis.get_ylim()) * 0.5, f'{value:.2f}',
                          rotation=0, verticalalignment='top', fontsize=10, color='black')
                prev_value = value

    # Plot idf_perc
    axes[0].plot(epochs, idf_perc_train, marker='o', linestyle='-', color='b', label='Train idf_perc')
    if correct_valid:
        axes[0].plot(epochs, idf_perc_valid, marker='o', linestyle='--', color='g', label='Valid idf_perc')
    axes[0].set_title(f'{title} IDF Percentage Over Epochs')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('idf_perc')
    axes[0].legend()
    axes[0].grid()
    annotate_changes(alphas, axes[0], 'Alpha')

   # Plot count_error
    axes[1].plot(epochs, count_error_train, marker='s', linestyle='-', color='r', label='Train count_error')
    if correct_valid:
        axes[1].plot(epochs, count_error_valid, marker='s', linestyle='--', color='m', label='Valid count_error')
    axes[1].set_title(f'{title} Count Error Over Epochs')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('count_error')
    axes[1].legend()
    axes[1].grid()
    #annotate_changes(learning_rates, axes[1], 'LR')
    annotate_changes(alphas, axes[1], 'Alpha')

    # Plot R2
    axes[2].plot(epochs, r2, marker='s', linestyle='-', color='r', label='Train')
    if correct_valid:
        axes[2].plot(epochs, r2_valid, marker='s', linestyle='--', color='m', label='Valid')
    axes[2].set_title(f'{title} R2 Over Epochs')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('R2')
    axes[2].legend()
    axes[2].grid()
    #annotate_changes(learning_rates, axes[2], 'LR')
    annotate_changes(alphas, axes[2], 'Alpha')
    plt.savefig(f'{title}.png')
    plt.tight_layout()
    plt.show()


In [ ]:
a = ['American Goldfinch', 'Black-capped Chickadee', 'Blue Jay', 'Chestnut-backed Scimitar Babbler', 'Mallard', 'Mourning Dove', 'Red-winged Blackbird', 'Rusty Blackbird']
b = ['American Goldfinch', 'American Tree Sparrow', 'Black-capped Chickadee', 'Blue Jay', 'Canada Goose', 'Dark-eyed Junco', 'Hairy Woodpecker', 'House Finch', 'Mallard', 'Mourning Dove', 'Northern Cardinal', 'Purple Finch', 'Song Sparrow', 'White-breasted Nuthatch', 'White-throated Sparrow']
cnt = [ 0.14705882966518402, 0.0714285746216774, 0.0, 0.0, 0.0, 0.6666666865348816, 0.0, 0.6666666865348816, 0.3333333432674408, 0.0, 0.0, 0.5, 0.6666666865348816, 0.6000000238418579, 0.8333333730697632, 0.30000001192092896, 0.375, 0.2857142984867096, 0.5714285969734192, 0.0, 0.4000000059604645, 0.0, 0.4000000059604645, 0.7777777910232544, 0.0, 0.625, 0.3333333432674408, 0.6000000238418579, 0.25, 0.0, 0.4285714626312256]
np.average(cnt)

In [ ]:

def new_alpha(alpha, changing_alpha, epoch, max_epoch, validation_loss):
  """
  changing alpha -
   0: no change
   1: increase every 2 epochs,
   2: decrease every 2 epochs,
   3: increase depending on eval loss function,
   4: decrease depending on eval loss functio
  """
  if(changing_alpha==0):
    return alpha, epoch<max_epoch
  if(epoch%2==0 and changing_alpha in [1,2]):
    if(changing_alpha==2):
      return alpha*0.8, epoch<max_epoch and alpha*0.8>0.1
    if(changing_alpha==1):
      if(alpha!=0):
        return alpha*1.2, epoch<max_epoch and alpha*1.2<0.9
      else:
        return 0.01+alpha*1.2, epoch<max_epoch
  if(epoch>2):
    if(alpha>0.5 and np.abs(validation_loss[-1]-validation_loss[-2])<20 and np.abs(validation_loss[-1]-validation_loss[-3])<20):
      if(changing_alpha==4):
        return alpha*0.85, epoch<max_epoch and alpha*0.85>0.05
      if(changing_alpha==2):
        if(alpha!=0):
          return alpha*1.2, epoch<max_epoch and alpha*1.2<0.9
        else:
          return 0.01+alpha*1.2, epoch<max_epoch
    if(alpha<0.5 and np.abs(validation_loss[-1]-validation_loss[-2])<50 and np.abs(validation_loss[-1]-validation_loss[-3])<50):
      if(changing_alpha==4):
        return alpha*0.85, epoch<max_epoch and alpha*0.85>0.05
      if(changing_alpha==2):
        if(alpha!=0):
          return alpha*1.2, epoch<max_epoch and alpha*1.2<0.9
        else:
          return 0.01+alpha*1.2, epoch<max_epoch
  return alpha, epoch<max_epoch


In [ ]:
def train_and_eval_mamba(model, train_loader, valid_loader, regular_bce, alpha, vector_dim =128, max_epochs = 10 , device = device, model_name="", changing_alpha=0, lr=5e-4):
    """
    Train and evaluate the model on a given dataset.

    Args:
        model: The trained model to evaluate.
        train_loader: DataLoader for the training dataset.
    returns:
        arr: array of losses for each epoch training
        arr: array of losses for each epoch validation
    """
    train_loss_arr = []
    valid_loss_arr = []
    correct_train =[]
    correct_valid = []
    idf_birds_perc = []  #the percentage of birds from the garget that were identified, every epoch we compute it from only one sequence
    epoch = 0
    min_count_error = np.inf
    csv_file_train = f'{model_name}_train.csv'
    csv_file_eval = f'{model_name}_eval.csv'
    run = True
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = CosineAnnealingLR(optimizer, T_max=max_epochs)
    train_mean = calculate_y_mean(train_loader)
    eval_mean = calculate_y_mean(valid_loader)
    while (run):
        torch.cuda.empty_cache()
        print(f"Epoch {epoch+1}")
        model, train_loss, correct = train_mamba(model, train_loader,  32, train_mean=train_mean, scheduler=scheduler, optimizer=optimizer, vector_dim =128, device = device, loss_type_weight = alpha, regular_bce=regular_bce)
        correct["epoch"] = epoch
        correct["alpha"] = alpha
        correct["learning_rate"] = optimizer.param_groups[0]['lr']
        if np.isnan(train_loss):
            print(f"NaN detected in loss at Epoch {epoch}. Stopping training.")
            break
        train_loss_arr.append(train_loss)
        correct_train.append(correct)
        df = pd.DataFrame([correct])
        df.to_csv(csv_file_train, mode='a', header=not pd.io.common.file_exists(csv_file_train), index=False)
        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} count accuracy {correct['count']}, bird prediction accuracy {correct['bird classifier']}, count_error: {correct['count_error']}")
        model , valid_loss, correct = eval_mamba(model, valid_loader,  32, vector_dim =128, val_mean=eval_mean, device = device, loss_type_weight = alpha)
        print(f"Epoch {epoch} - Valid Loss: {valid_loss:.4f} count accuracy {correct['count']}, bird prediction accuracy {correct['bird classifier']}, count_error: {correct['count_error']}, , alpha: {alpha}")
        correct["epoch"] = epoch
        correct["alpha"] = alpha
        correct["learning_rate"] = optimizer.param_groups[0]['lr']
        if np.isnan(valid_loss):
            print(f"NaN detected in loss at Epoch {epoch}. Stopping training.")
            break
        valid_loss_arr.append(valid_loss)
        correct_valid.append(correct)
        df = pd.DataFrame([correct])
        df.to_csv(csv_file_eval, mode='a', header=not pd.io.common.file_exists(csv_file_eval), index=False)
        epoch+=1
        alpha, run = new_alpha(alpha, changing_alpha, epoch, max_epochs,valid_loss_arr )
        scheduler.step()
        if(correct["count_error"]<min_count_error):
          min_count_error = correct["count_error"]
          torch.save(model.state_dict(), f"model_{model_name}.pth")
    return train_loss_arr, valid_loss_arr, correct_train, correct_valid, model



In [ ]:
"""
alphas = [ 0, 1, 0.3]
for a in alphas:
  # Instantiate model and move to the correct device
  model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
  model.apply(init_weights)
  train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{a}_BCE_Loss", alpha=a,regular_bce=True, lr=2e-4)
  plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"alpha={a}_BCE")
  model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
  model.apply(init_weights)
  train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{a}_Facal_Loss", alpha=a,regular_bce=False, lr=2e-4)
  plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"alpha={a}_Facal")
"""

In [ ]:
#Run changing alphas for decrease alpha (starting from alpha=1) and increase alpha (starting from alpha=0)
"""
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{1}_BCE_Loss_decreasing_dynamic_epochs", alpha=1,regular_bce=True, lr=2e-4, changing_alpha=4)
plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"decrease alpha dynamicly - BCE Loss")
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{1}_BCE_Loss_decreasing_every_2_epochs", alpha=1,regular_bce=True, lr=2e-4, changing_alpha=2)
plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"decrease alpha every 2 epochs - BCE Loss")
#Run alpha=1
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{1}_BCE_Loss", alpha=1,regular_bce=True, lr=2e-4)
plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"alpha={1}_BCE")
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{1}_Focal_Loss", alpha=1,regular_bce=False, lr=2e-4)
plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"alpha={1}_Focal")
"""
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{0}_BCE_Loss_increase_every_2_epochs", alpha=0,regular_bce=True, lr=2e-4, changing_alpha=1)
plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"increase alpha every 2 epochs - BCE Loss")
#Run changing alphas
model = MambaBirdClassifier(device=device, n_classes=len(avi_set), seq_len=1700).to(device)
model.apply(init_weights)
train_loss_arr, valid_loss_arr, correct_train, correct_valid, trained_model = train_and_eval_mamba(model, train_loader, valid_loader, model_name=f"final_alpha:{0}_BCE_Loss_increase_dynamic_epochs", alpha=0,regular_bce=True, lr=2e-4, changing_alpha=3)
plot_epoch_metrics_with_alpha_and_learning_rate(correct_train, correct_valid,  title=f"increase alpha every 2 epochs - BCE Loss")

In [ ]:
plt.plot(train_loss_arr, label='Training Loss')
plt.plot(valid_loss_arr, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()

plt.plot([correct_train_arr[i]["count"] for i in range(len(correct_train_arr))], label='Training')
plt.plot([correct_valid_arr[i]["count"] for i in range(len(correct_valid_arr))], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Count Accuracy')
plt.title('Training and Validation Count Accuracy Over Epochs')
plt.legend()
plt.show()

plt.plot([correct_train_arr[i]['bird classifier'] for i in range(len(correct_train_arr))], label='Training')
plt.plot([correct_valid_arr[i]['bird classifier'] for i in range(len(correct_valid_arr))], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Classification Accuracy')
plt.title('Training and Validation Classification Accuracy Over Epochs')
plt.legend()
plt.show()

In [ ]:
import torch
from sklearn.metrics import accuracy_score, classification_report

def evaluate_model(model, test_loader, device, threshold=0.5):
    """
    Evaluate the model on the test set and compute accuracy & classification report.

    Args:
        model: Trained model.
        test_loader: DataLoader for the test dataset.
        device: The device (CPU/GPU) to run the evaluation on.
        threshold: The threshold for multi-label classification (default=0.5).

    Returns:
        accuracy: The accuracy of the model on the test dataset.
        classification_report: Detailed classification report.
    """
    model.eval()  # Set model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for feats, target, ids in test_loader:
            feats, target = feats.to(device), target.to(device)

            # Forward pass through model
            logits = model(feats)  # Get model outputs
            probs = torch.sigmoid(logits)  # Apply sigmoid activation

            # Apply threshold to get binary predictions
            preds = (probs > threshold).int()

            # Move predictions and labels to CPU
            preds_np = preds.cpu().numpy()
            target_np = target.cpu().numpy()

            # Collect all predictions and labels
            all_preds.append(preds_np)
            all_labels.append(target_np)

            # Print example predictions
            for i in range(len(preds_np)):
                decoded_pred = decode_birds(preds_np[i], bird_to_index)  # Decoded prediction
                decoded_target = decode_birds(target_np[i], bird_to_index)  # Decoded target
                print(f"Item {i}:")
                print(f"Predicted Birds: {decoded_pred}")
                print(f"Target Birds: {decoded_target}")

    # Convert list of batches into a single numpy array
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Compute accuracy and classification report
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=[str(i) for i in range(all_labels.shape[1])])

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("Classification Report:")
    print(report)

    return accuracy, report

# After training
evaluate_model(model, test_loader, device)
